In [1]:
from datasets import load_dataset

data = load_dataset("universal_dependencies", "ar_padt")

Found cached dataset universal_dependencies (/home/magedsaeed/.cache/huggingface/datasets/universal_dependencies/ar_padt/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 6075
    })
    validation: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 909
    })
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 680
    })
})

In [3]:
TAGS = [
    "NOUN",
    "PUNCT",
    "ADP",
    "NUM",
    "SYM",
    "SCONJ",
    "ADJ",
    "PART",
    "DET",
    "CCONJ",
    "PROPN",
    "PRON",
    "X",
    "_",  # we need to drop this class is it is meant to refer to unsplit words. For instance, the word وكانت is split into و and كانت but it is assigned this class as a whole.
    "ADV",
    "INTJ",
    "VERB",
    "AUX",
]
tag_names_to_labels = {tag_name: tag_label for tag_label, tag_name in enumerate(TAGS)}
tag_labels_to_names = {
    tag_label: tag_name for tag_name, tag_label in tag_names_to_labels.items()
}


In [4]:
# prompt without fewshots
sys_msg = """
I wish you can generate a table of Arabic POS tags following Universal Dependencies tagset in the following format:
TOKEN:POS
Please note that I tokenized the sentence for you. Please do not change, add, combine, merge or remove any of these tokens such as ب and ه. Please consider punctuation marks as separate tokens, always. Split them as two separate tokens if they come together and classify each of them independently.
Please give me the generated table and that is it. No further discussion, explanation or extrapolation required.
""".strip()


# prompt with fewshots
# sys_msg_with_fewshots = lambda fewshots: """
# I wish you can generate a table of Arabic POS tags following Universal Dependencies tagset in the following format:
# WORD  |  POS
# Please do not forget to split punctuation marks such as quotes and classify them.
# Please, also, do not forget to split words that have CCONJ tokens such as و and ف.
# Please give me the generated table and that is it. No further discussion, explanation or extrapolation required.
# here are some examples to consider:
# """+'\n'.join(f"""
# sentence: {sentence}
# tags:
# {tags}""" for (sentence,tags) in fewshots)+"""

# Now, My sentence is:
# {input_sentence}
# """


In [5]:
def construct_tags_map_str(sentence_tokens, tags):
    assert len(sentence_tokens)==len(tags), "lengths of the tokens and its tags does not match!"
    tags_map = ""
    for word, tag_label in zip(sentence_tokens, tags):
        if tag_label == 13:  # see why in the comment in TAGS list
            continue
        tags_map += f"{word}:{tag_labels_to_names[tag_label]}"
        tags_map += "\n"
    return tags_map

In [6]:
# testing fewshots prompt
# sentences = [
#     'السلام عليكم ورحمة الله',
#     'كيف الحال'
# ]
# fewshots = [(
#     sentence,
#     construct_tags_map_str(sentence,[random.choice(range(len(TAGS))) for _ in sentence.split()])
# )for sentence in sentences]
# print(sys_msg_with_fewshots(fewshots).format(input_sentence="وكانت خسائر المجموعة الاسبانية الرسمية تراكمت في السنوات العشر الاخيرة لتبلغ 920 مليون يورو"))


In [7]:
def create_chat_prompt(input_text):
    return [
        {"role": "system", "content": sys_msg},
        {"role": "user", "content": input_text},
    ]


def create_chat_example(sentence, tags):
    return [
        {"role": "system", "content": sentence, "name": "example_user"},
        {"role": "system", "content": tags, "name": "example_assistant"},
    ]


In [8]:
!mkdir -p ../registry/data/padt

In [9]:
small_samples_data = data["train"].filter(lambda example: len(example['tokens']) < 100)
small_samples_data

Loading cached processed dataset at /home/magedsaeed/.cache/huggingface/datasets/universal_dependencies/ar_padt/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7/cache-c7383425e2cd321a.arrow


Dataset({
    features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
    num_rows: 5774
})

In [10]:
small_samples_data = data['train'].to_pandas()
small_samples_data = small_samples_data.loc[small_samples_data['tokens'].map(len) < 50]
small_samples_data

,idx,text,tokens,lemmas,upos,xpos,feats,head,deprel,deps,misc
0,afp.20000715.0075:p1u1,برلين ترفض حصول شركة اميركية على رخصة تصنيع دب...,"[برلين, ترفض, حصول, شركة, اميركية, على, رخصة, ...","[بَرلِين, رَفَض, حُصُول, شَرِكَة, أَمِيرِكِيّ,...","[12, 16, 0, 0, 6, 2, 0, 0, 0, 1, 12, 1, 6]","[X---------, VIIA-3FS--, N------S4R, N------S2...","[{'Foreign': 'Yes'}, {'Aspect': 'Imp', 'Gender...","[2, 0, 2, 3, 4, 7, 3, 7, 8, 11, 9, 11, 9]","[nsubj, root, obj, nmod, amod, case, obl:arg, ...","[[('nsubj', 2)], [('root', 0)], [('obj', 2)], ...","[{'Vform': 'بَرلِين', 'Gloss': 'Berlin', 'Root..."
2,afp.20000715.0075:p3u1,وفي نيسان/ابريل الماضي، تخلت الدولة الاسبانية ...,"[وفي, و, في, نيسان, /, ابريل, الماضي, ،, تخلت,...","[_, وَ, فِي, نِيسَان, /, أَبرِيل, مَاضِي, ،, ت...","[13, 9, 2, 0, 1, 0, 6, 1, 16, 0, 6, 2, 0, 1, 1...","[None, C---------, P---------, N------S2I, G--...","[None, None, {'AdpType': 'Prep'}, {'Case': 'Ge...","[None, 0, 3, 8, 5, 3, 3, 6, 1, 8, 9, 12, 8, 15...","[_, root, case, obl, punct, appos, amod, punct...","[None, [('root', 0)], [('case', 3)], [('obl:فِ...","[None, {'Vform': 'وَ', 'Gloss': 'and', 'Root':..."
3,afp.20000715.0075:p4u1,وكانت خسائر المجموعة الاسبانية الرسمية تراكمت ...,"[وكانت, و, كانت, خسائر, المجموعة, الاسبانية, ا...","[_, وَ, كَان, خَسَارَة, مَجمُوعَة, إِسبَانِيّ,...","[13, 9, 16, 0, 0, 6, 6, 16, 2, 0, 3, 6, 13, 9,...","[None, C---------, VP-A-3FS--, N------P1R, N--...","[None, None, {'Aspect': 'Perf', 'Gender': 'Fem...","[None, 0, 1, 2, 3, 4, 3, 2, 9, 7, 9, 9, None, ...","[_, root, parataxis, nsubj, nmod, amod, amod, ...","[None, [('root', 0)], [('parataxis', 1)], [('n...","[None, {'Vform': 'وَ', 'Gloss': 'and', 'Root':..."
4,afp.20000715.0075:p5u1,واشارت صحيفة الاحد الى ان المستشار شرودر يعتبر...,"[واشارت, و, أشارت, صحيفة, الاحد, الى, ان, المس...","[_, وَ, أَشَار, صَحِيفَة, أَحَد, إِلَى, أَنَّ,...","[13, 9, 16, 0, 0, 2, 5, 0, 12, 16, 5, 0, 0, 6,...","[None, C---------, VP-A-3FS--, N------S1R, N--...","[None, None, {'Aspect': 'Perf', 'Gender': 'Fem...","[None, 0, 1, 2, 3, 9, 9, 9, 7, 2, 19, 19, 11, ...","[_, root, parataxis, nsubj, nmod, mark, mark, ...","[None, [('root', 0)], [('parataxis', 1)], [('n...","[None, {'Vform': 'وَ', 'Gloss': 'and', 'Root':..."
5,afp.20000715.0075:p6u1,وتعذر لمتحدثة باسم وزارة الدفاع الالمانية ان ت...,"[وتعذر, و, تعذر, لمتحدثة, ل, متحدثة, باسم, ب, ...","[_, وَ, تَعَذَّر, _, لِ, مُتَحَدِّثَة, _, بِ, ...","[13, 9, 16, 13, 2, 0, 13, 2, 0, 0, 0, 6, 5, 16...","[None, C---------, VP-A-3MS--, None, P--------...","[None, None, {'Aspect': 'Perf', 'Gender': 'Mas...","[None, 0, 1, None, 4, 2, None, 7, 5, 4, 7, 7, ...","[_, root, parataxis, _, case, obl:arg, _, case...","[None, [('root', 0)], [('parataxis', 1)], None...","[None, {'Vform': 'وَ', 'Gloss': 'and', 'Root':..."
...,...,...,...,...,...,...,...,...,...,...,...
6068,xinhua.20030511.0192:p1u1,تنسيق المواقف بين مصر وفلسطين قبيل زيارة باول ...,"[تنسيق, المواقف, بين, مصر, وفلسطين, و, فلسطين,...","[تَنسِيق, مَوقِف, بَينَ, مصر, _, وَ, فِلَسطِين...","[0, 0, 2, 12, 13, 9, 0, 2, 0, 12, 2, 0]","[N------S1R, N------P2D, PI------4-, U--------...","[{'Case': 'Nom', 'Definite': 'Cons', 'Number':...","[0, 1, 4, 1, None, 6, 4, 8, 1, 8, 11, 8]","[root, nmod, case, obl:arg, _, cc, conj, case,...","[[('root', 0)], [('nmod:gen', 1)], [('case', 4...","[{'Vform': 'تَنسِيقُ', 'Gloss': 'coordination,..."
6069,xinhua.20030511.0192:p2u1,القاهرة 11 مايو,"[القاهرة, 11, مايو]","[قَاهِرَة, 11, مَايُو]","[0, 3, 0]","[N------S1D, Q---------, N------S2I]","[{'Case': 'Nom', 'Definite': 'Def', 'Number': ...","[0, 1, 2]","[root, dep, nmod]","[[('root', 0)], [('dep', 1)], [('nmod:gen', 2)]]","[{'Vform': 'اَلقَاهِرَةُ', 'Gloss': 'Cairo', '..."
6072,xinhua.20030511.0192:p5u1,واكد شعث فى الاتصال على ضرورة قبول الجانب الإس...,"[واكد, و, أكد, شعث, فى, الاتصال, على, ضرورة, ق...","[_, وَ, أَكَّد, شعث, فِي, اِتِّصَال, عَلَى, ضَ...","[13, 9, 16, 12, 2, 0, 2, 0, 0, 0, 6, 0, 12, 0,...","[None, C---------, VP-A-3MS--, U---------, P--...","[None, None, {'Aspect': 'Perf', 'Gende

In [11]:
dev_df = small_samples_data
dev_df["sample"] = dev_df.apply(
    lambda x: create_chat_example(
        sentence=' '.join([token for token,tag in zip(x["tokens"],x["upos"]) if tag != 13]),
        tags=construct_tags_map_str(sentence_tokens=x["tokens"], tags=x["upos"]),
    ),
    axis=1,
)
dev_df[["sample"]].to_json(
    "../registry/data/padt/few_shot.jsonl",
    lines=True,
    orient="records",
    force_ascii=False,
)

test_df = data["test"].to_pandas()

test_df["input"] = test_df.apply(
    lambda row: create_chat_prompt(
        input_text=' '.join([token for token,tag in zip(row["tokens"],row["upos"]) if tag != 13])
    ),
    axis=1,
)
test_df["ideal"] = test_df.apply(
    lambda row: construct_tags_map_str(
        sentence_tokens=row["tokens"],
        tags=row["upos"],
    ),
    axis=1,
)
test_df[["input", "ideal"]].to_json(
    "../registry/data/padt/samples.jsonl",
    lines=True,
    orient="records",
    force_ascii=False,
)


In [57]:
import os

os.environ["OPENAI_API_KEY"] = "<openai-key>"
os.environ["EVALS_THREADS"] = "3"
os.environ["EVALS_THREAD_TIMEOUT"] = "600"


In [13]:
!ls

ajgt.ipynb  ar-constants  metrec.ipynb	tashkeela.ipynb
apb.ipynb   easc.ipynb	  padt.ipynb	un_v1.ipynb


In [14]:
!ls ../registry/data/padt

few_shot.jsonl	samples.jsonl


In [15]:
pos_tagging_task_specs = """
padt:
    id: padt.test.v1
    metrics: [accuracy]
    description: Evaluate Arabic POS tagging
# Define the eval
padt.test.v1:
  # Specify the class name as a dotted path to the module and class
  class: evals.elsuite.pos_tagging:POSTagger
  args:
    samples_jsonl: padt/samples.jsonl
    few_shot_jsonl: padt/few_shot.jsonl # path to few shots file
    num_few_shot: 0 # max few shots to use

""".strip()
with open("../registry/evals/padt.yaml", "w") as file:
    file.write(pos_tagging_task_specs)


In [59]:
record_path = "../../evals/eval_results/padt_with_max_samples_fixed_accuracy_spaces_bug.jsonl"
# !oaieval gpt-3.5-turbo-0301 padt --record_path {record_path} --seed 41 --max_samples 1
# !oaieval gpt-3.5-turbo-0301 padt --record_path {record_path} --seed 41 --max_samples 10 --modelspec_extra_options temperature=0
!oaieval gpt-3.5-turbo-0301 padt --record_path {record_path} --seed 41 --modelspec_extra_options temperature=0.0 --max_samples 680

[2023-05-13 19:12:09,683] [registry.py:156] Loading registry from /home/magedsaeed/MyProjects/evals/evals/registry/evals
[2023-05-13 19:12:09,791] [registry.py:156] Loading registry from /home/magedsaeed/.evals/evals
[2023-05-13 19:12:10,795] [oaieval.py:213] Run started: 230513161210NUXQLVJB
[2023-05-13 19:12:10,802] [data.py:75] Fetching padt/samples.jsonl
[2023-05-13 19:12:10,854] [eval.py:32] Evaluating 680 samples
[2023-05-13 19:12:10,861] [eval.py:152] Running in threaded mode with 3 threads!
 20%|███████▋                               | 134/680 [22:18<2:03:19, 13.55s/it][2023-05-13 19:34:29,325] [_common.py:105] Backing off openai_chat_completion_create_retrying(...) for 0.7s (openai.error.RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7ba2ed96b4c07b3474bb72bab8a6e0dd in your message.))
[2023-05-13 19:35:00,414] [_com

In [63]:
import pandas as pd

with open(record_path, "r") as f:
    events_df = pd.read_json(f, lines=True)


In [64]:
events_df[events_df["final_report"].notnull()]["final_report"]


2001    {'accuracy': 0.7591028108921981}
Name: final_report, dtype: object

In [65]:
# matches_df = events_df[events_df.type == "tagging"].reset_index(drop=True)
# matches_df = matches_df.join(pd.json_normalize(matches_df.data))
# matches_df.accuracy.value_counts().plot.bar(
#     title="Correctness of generated answers", xlabel="Correctness", ylabel="Count"
# )

In [66]:
for i, r in pd.json_normalize(events_df[events_df.type == "sampling"].data).iterrows():
    print(f"Prompt: {r.prompt[-1]}")
    print(f"Sampled: {r.sampled}")
    print(f"{i}", "--" * 25)


Prompt: {'role': 'user', 'content': 'سوريا : تعديل وزاري واسع يشمل 8 حقائب'}
Sampled: سوريا:NOUN
: :PUNCT
تعديل:NOUN
وزاري:ADJ
واسع:ADJ
يشمل:VERB
8:NUM
حقائب:NOUN
0 --------------------------------------------------
Prompt: {'role': 'user', 'content': 'و ذكرت وكالة الانباء السورية ان التعديل شمل ثماني حقائب بين ها وزارتا الداخلية و الاقتصاد .'}
Sampled: و:CCONJ
ذكرت:VERB
وكالة:NOUN
الانباء:NOUN
السورية:ADJ
ان:SCONJ
التعديل:NOUN
شمل:VERB
ثماني:NUM
حقائب:NOUN
بين:ADP
ها:PRON
وزارتا:NOUN
الداخلية:NOUN
و:CCONJ
الاقتصاد:NOUN
.:PUNCT
1 --------------------------------------------------
Prompt: {'role': 'user', 'content': 'دمشق ( وكالات الانباء ) - اجرى الرئيس السوري بشار الاسد تعديلا حكومياً واسعا تم ب موجب ه إقالة وزيري الداخلية و الإعلام عن منصبي ها في حين ظل محمد ناجي العطري رئيساً ل الحكومة .'}
Sampled: دمشق:NOUN
(:PUNCT
وكالات:NOUN
ال:DET
انباء:NOUN
):PUNCT
-:PUNCT
اجرى:VERB
ال:DET
رئيس:NOUN
ال:DET
سوري:ADJ
بشار:PROPN
الاسد:PROPN
تعديلا:NOUN
حكومياً:ADV
واسعا:ADJ
تم:VERB
ب:PUNCT
موجب:NO

In [67]:
for i, r in pd.json_normalize(events_df[events_df.type == "tagging"].data).iterrows():
    print(f"Prompt: {r.prompt[-1]}")
    print(f"Sampled: {r.sampled}")
    print(f"{i}", "--" * 25)

AttributeError: 'Series' object has no attribute 'prompt'

In [68]:
len(sorted([int(i.split('.')[-1]) for i in events_df[events_df.type == "tagging"]['sample_id']]))

680

In [69]:
for  j in sorted([int(i.split('.')[-1]) for i in events_df[events_df.type == "tagging"]['sample_id']]):
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [51]:
tagging_events = events_df[events_df.type == "tagging"]
tagging_events[tagging_events['sample_id']=='padt.test.122'].data[408]

{'accuracy': 0.872549019607843,
 'sampled': 'يبدو:VERB\nأن:SCONJ\nالقدر:NOUN\nأراد:VERB\nأن:SCONJ\nيظل:VERB\nالطيار:NOUN\nالمصري:ADJ\nعلي:ADP\nمراد:NOUN\nرمزا:NOUN\nل:ADP\nعداء:NOUN\nالمصريين:NOUN\nل:ADP\nكل:DET\nما:PRON\nهو:PRON\nإسرائيلي:ADJ\n.:PUNCT\nف:SCONJ\nالضجة:NOUN\nالتي:PRON\nأثار:VERB\nها:PRON\nمراد:NOUN\nلم:NOUN\nتنته:VERB\nبعد:PUNCT\n،:PUNCT\nعلى:ADP\nرغم:ADP\nمرور:NOUN\nشهور:NOUN\nعدة:ADJ\nعلى:ADP\nواقعة:NOUN\nرفض:NOUN\nه:PRON\nقيام:NOUN\nجنود:NOUN\nإسرائيليين:ADJ\nب:ADP\nتفتيش:NOUN\nطائرة:NOUN\nتابعة:ADJ\nل:ADP\nشركة:NOUN\n»:PUNCT\nمصر:NOUN\nل:ADP\nالطيران:NOUN\n«:PUNCT\nحطّ:VERB\nب:ADP\nها:PRON\nفي:ADP\nمطار:NOUN\nغزة:NOUN\nفي:ADP\nايلول:NOUN\n(:PUNCT\nسبتمبر:NOUN\n):PUNCT\nمن:ADP\nالعام:NOUN\nالماضي:ADJ\n.:PUNCT\nو:CCONJ\nتحول:VERB\nالطيار:NOUN\nالى:ADP\nبطل:NOUN\nشعبي:ADJ\nبعدما:SCONJ\nأخضعت:VERB\nه:PRON\nشركة:NOUN\n»:PUNCT\nمصر:NOUN\nل:ADP\nالطيران:NOUN\n«:PUNCT\nل:ADP\nالتحقيق:NOUN\nو:CCONJ\nأوقفت:VERB\nه:PRON\nعن:ADP\nالعمل:NOUN\nثم:ADV\nأحالت:VERB\nه:PRON\nعلى:ADP\

In [53]:
tagging_events[tagging_events['sample_id']=='padt.test.122'].data[432]

{'accuracy': 0.833333333333333,
 'sampled': 'يبدو:VERB\nأن:SCONJ\nالقدر:NOUN\nأراد:VERB\nأن:SCONJ\nيظل:VERB\nالطيار:NOUN\nالمصري:ADJ\nعلي:ADP\nمراد:NOUN\nرمزا:NOUN\nل:ADP\nعداء:NOUN\nالمصريين:NOUN\nل:ADP\nكل:DET\nما:PRON\nهو:PRON\nإسرائيلي:ADJ\n.:PUNCT\nف:SCONJ\nالضجة:NOUN\nالتي:PRON\nأثار:VERB\nها:PRON\nمراد:NOUN\nلم:NOUN\nتنتهي:VERB\nبعد:PUNCT\n،:PUNCT\nعلى:ADP\nرغم:ADP\nمرور:NOUN\nشهور:NOUN\nعدة:ADJ\nعلى:ADP\nواقعة:NOUN\nرفض:NOUN\nه:PRON\nقيام:NOUN\nجنود:NOUN\nإسرائيليين:ADJ\nب:PUNCT\nتفتيش:NOUN\nطائرة:NOUN\nتابعة:ADJ\nل:ADP\nشركة:NOUN\n»:PUNCT\nمصر:NOUN\nل:PUNCT\nالطيران:NOUN\n«:PUNCT\nحطّ:VERB\nب:PUNCT\nها:PRON\nفي:ADP\nمطار:NOUN\nغزة:NOUN\nفي:ADP\nايلول:NOUN\n(:PUNCT\nسبتمبر:NOUN\n):PUNCT\nمن:ADP\nالعام:NOUN\nالماضي:ADJ\n.:PUNCT\nو:CCONJ\nتحول:VERB\nالطيار:NOUN\nالى:ADP\nبطل:NOUN\nشعبي:ADJ\nبعدما:SCONJ\nأخضعت:VERB\nه:PRON\nشركة:NOUN\n»:PUNCT\nمصر:NOUN\nل:PUNCT\nالطيران:NOUN\n«:PUNCT\nل:ADP\nالتحقيق:NOUN\nو:CCONJ\nأوقفت:VERB\nه:PRON\nعن:ADP\nالعمل:NOUN\nثم:ADV\nأحالت:VERB\nه:PRON\

In [52]:
tagging_events[tagging_events['sample_id']=='padt.test.122']

,spec,run_id,event_id,sample_id,type,data,created_by,created_at
408,NaN,230512061210P6ZP52PS,407.0,padt.test.122,tagging,"{'accuracy': 0.872549019607843, 'sampled': 'يب...",,2023-05-12 06:26:37.071275+00:00
432,NaN,230512061210P6ZP52PS,431.0,padt.test.122,tagging,"{'accuracy': 0.833333333333333, 'sampled': 'يب...",,2023-05-12 06:28:08.192224+00:00
462,NaN,230512061210P6ZP52PS,461.0,padt.test.122,tagging,"{'accuracy': 0.8382352941176471, 'sampled': 'ي...",,2023-05-12 06:30:00.874356+00:00
480,NaN,230512061210P6ZP52PS,479.0,padt.test.122,tagging,"{'accuracy': 0.848039215686274, 'sampled': 'يب...",,2023-05-12 06:31:37.306904+00:00
508,NaN,230512061210P6ZP52PS,507.0,padt.test.122,tagging,"{'accuracy': 0.848039215686274, 'sampled': 'يب...",,2023-05-12 06:33:20.730777+00:00
...,...,...,...,...,...,...,...,...
4555,NaN,230512061210P6ZP52PS,4554.0,padt.test.122,tagging,"{'accuracy': 0.8137254901960781, 'sampled': 'ي...",,2023-05-12 13:24:55.759384+00:00
4564,NaN,230512061210P6ZP52PS,4563.0,padt.test.122,tagging,"{'accuracy': 0.8382352941176471, 'sampled': 'ي...",,2023-05-12 13:26:31.404151+00:00
4573,NaN,230512061210P6ZP52PS,4572.0,padt.test.122,tagging,"{'accuracy': 0.848039215686274, 'sampled': 'يب...",,2023-05-12 13:28:12.170796+00:00
4583,NaN,230512061210P6ZP52PS,4582.0,padt.test.122,tagging,"{'accuracy': 0.8284313725490191, 'sampled': 'ي...",,2023-05-12 13:30:03.797965+00:00
